# Stitching a VBI Moasic with `reproject`

The [reproject](https://reproject.readthedocs.io/) package is an Astropy-affiliated package for regridding data.
A number of different algorithms are implemented in the package, with different trade-offs for speed and accuracy.
Reprojecting a single spatial image such as an AIA image is well supported and demonstrated in the [sunpy gallery](https://docs.sunpy.org/en/latest/generated/gallery/index.html#combining-co-aligning-and-reprojecting-images).
When working with DKIST data, there are normally many spatial frames inside a dataset, and reprojecting them all can be very computationally expensive.

We are going to use the example of using reproject's {obj}`reproject.mosaicking.reproject_and_coadd` function to stitch a mosaic of VBI frames, in a way which could be extended to be flux conserving.


## Obtaining some data

Let's download a VBI dataset, which has good seeing:

This gives us a `TiledDataset` object, which as we have already seen is like an array of `Dataset` objects.

This mosaic is 3 x 3:

To start off, let's just download the first frame of each tile.

We can now make a composite plot of all the tiles.

## Regridding with Reproject

First, we need the most recent version of reproject (released on Friday) for this to work, so let's check:

Now import everything we need for it to run

First, let us crop off the edges of all our tiles to remove some artifacts.

Next we need to calculate the optimal WCS for the output:

Now we can do the actual reprojection

Make a new `NDCube` object and plot it.

We can see some small artifacting which can be hidden by using the `match_background=True` keyword argument to `reproject_and_coadd`.